# run this first to set up the libraries

In [ ]:
import os
os.chdir('/src/notebooks')
import lib.postgres as psql
con, cur = psql.connect_to_postgres()
from redis import Redis
from rq import Queue
import lib.playgames as play
import ipywidgets as widgets
import numpy as np
REDIS = Redis(host='this_redis')
Q = Queue(connection=REDIS)
import io
import numpy as np
import chess
import chess.svg
import chess.polyglot
import ipywidgets as widgets
from PIL import Image as PImage

def numpyImage(path,dtype = "float32"):
    orig = PImage.open(path)
    originalPage = np.asarray(orig.convert("L"), dtype=np.float32) if dtype == "float32" else np.array(orig)
    orig.close()
    return originalPage

def makeImages(game):
    pgn = io.StringIO(game)
    game = chess.pgn.read_game(pgn)
    board = chess.Board()
    images = []
    zobrist = []
    images.append(chess.svg.board(board,size=200,coordinates=False))
    zobrist.append(chess.polyglot.zobrist_hash(board))
    if str(zobrist[0]) not in [ x.replace(".png","") for x in os.listdir("/src/notebooks/data/images/") if ".png" in x ]:
        !inkscape -z -e /src/notebooks/data/images/{zobrist[0]}.png -w 256 -h 256 /src/notebooks/data/images/{zobrist[0]}.svg 2>/dev/null 1>/dev/null
    for move in game.mainline_moves():
        board.push(move)
        images.append(chess.svg.board(board,size=200,coordinates=False))
        zobrist.append(chess.polyglot.zobrist_hash(board))
    for i,image in enumerate(images):
        f = open("/src/notebooks/data/images/"+str(zobrist[i])+".svg", 'w')
        f.write(image)
        if str(zobrist[i]) not in [ x.replace(".png","") for x in os.listdir("/src/notebooks/data/images/") if ".png" in x ]:
            !inkscape -z -e /src/notebooks/data/images/{zobrist[i]}.png -w 256 -h 256 /src/notebooks/data/images/{zobrist[i]}.svg 2>/dev/null 1>/dev/null
        f.close()
    X = np.zeros((256,256,len(zobrist)))
    for i,z in enumerate(zobrist):
        X[:,:,i] = numpyImage("/src/notebooks/data/images/"+str(zobrist[i])+".png")
    return X

# suppose you want to see leela and stockfish play the King's gambit, just put the opening moves in here and run the playGame function...

In [ ]:
line = "1. e4 e5 2. f4 exf4 3. Bc4"
gameOne = Q.enqueue(play.playGame,10,line,"leela","stockfish",job_timeout=600000)
gameTwo = Q.enqueue(play.playGame,10,line,"stockfish","leela",job_timeout=600000)

# or the Danish gambit

In [ ]:
line = "1. e4 e5 2. d4 exd4 3. c3 dxc3 4. Bc4 cxb2 5. Bxb2"
gameOne = Q.enqueue(play.playGame,10,line,"leela","stockfish",job_timeout=600000)
gameTwo = Q.enqueue(play.playGame,10,line,"stockfish","leela",job_timeout=600000)

# After you launch the games you can go to 127.0.0.1:5000 or something similar to see job running. (the background jobs seems to be working now)

In [ ]:
cur.execute("select * from games order by game desc limit 8")
games = cur.fetchall()
for i,game in enumerate(games):
    print(i,game)
    print("\n")

# added a really sloppy inline game viewer. Its a work in progress for sure. Right now it saves png files to the data directory and reads them in with numpy. This process is too slow to be practical but it's a start.

In [ ]:
game_I_want_to_View = 5 # pick from on of the rows in the previous cell
%matplotlib inline
%matplotlib notebook
import ipywidgets as widgets
import matplotlib.pyplot as plt
import lib.pieceapp as app
moves = games[game_I_want_to_View][6]
white = games[game_I_want_to_View][1]
black = games[game_I_want_to_View][2]
bs = app.IndexTracker(makeImages(moves),white,black)